In [1]:
import gcsfs
import cftime
import intake
from cmip6_preprocessing.preprocessing import combined_preprocessing
from fastjmd95 import rho
from xgcm import Grid
import numpy as np
import xesmf as xe
import xarray as xr
import pandas as pd
import cmocean as cmo

### Download CMIP6 data and regrid with CFSR data

In [2]:
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6-noQC.json"
col = intake.open_esm_datastore(cat_url)
cat = col.search(table_id='Omon', 
                 experiment_id=['historical'],
                 variable_id=['thetao'],
                 member_id=['r1i1p1f1'],
                 grid_label='gn',
                 source_id=['MPI-ESM1-2-HR']
                )

cmip6_compiled = cat.to_dataset_dict(
            zarr_kwargs={'consolidated':True, 'decode_times': True, 'use_cftime': True},
            preprocess=combined_preprocessing
            )


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


/Users/odyssey/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/Users/odyssey/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self

In [3]:
URL = 'https://ds.nccs.nasa.gov/thredds/dodsC/bypass/CREATE-IP/reanalysis/CFSR/mon/ocean/thetao.ncml'
CFSR_thetao = xr.open_dataset(URL,engine='netcdf4')

In [4]:
ds = cmip6_compiled['CMIP.MPI-M.MPI-ESM1-2-HR.historical.Omon.gn']
ds

<xarray.Dataset>
Dimensions:        (bnds: 2, lev: 40, member_id: 1, time: 1980, vertex: 4, x: 802, y: 404)
Coordinates: (12/15)
  * x              (x) float64 0.9539 1.404 1.854 2.304 ... 358.3 359.2 359.6
  * y              (y) float64 -78.67 -78.28 -77.89 -77.5 ... 89.42 89.52 89.76
    lat            (y, x) float64 dask.array<chunksize=(404, 802), meta=np.ndarray>
  * lev            (lev) float64 6.0 17.0 27.0 ... 4.67e+03 5.17e+03 5.72e+03
    lev_bounds     (lev, bnds) float64 dask.array<chunksize=(40, 2), meta=np.ndarray>
    lon            (y, x) float64 dask.array<chunksize=(404, 802), meta=np.ndarray>
    ...             ...
    lon_verticies  (y, x, vertex) float64 dask.array<chunksize=(404, 802, 4), meta=np.ndarray>
  * bnds           (bnds) int64 0 1
  * vertex         (vertex) int64 0 1 2 3
    lon_bounds     (bnds, y, x) float64 dask.array<chunksize=(1, 404, 802), meta=np.ndarray>
    lat_bounds     (bnds, y, x) float64 dask.array<chunksize=(1, 404, 802), meta=np.ndarray>
  * member_id      (member_id) <U8 'r1i1p1f1'
Data variables:
    thetao         (member_id, time, lev, y, x) float32 dask.array<chunksize=(1, 2, 40, 404, 802), meta=np.ndarray>
Attributes: (12/52)
    Conventions:             CF-1.7 CMIP-6.2
    activity_id:             CMIP
    branch_method:           standard
    branch_time_in_child:    0.0
    branch_time_in_parent:   0.0
    cmor_version:            3.5.0
    ...                      ...
    variable_id:             thetao
    variant_label:           r1i1p1f1
    netcdf_tracking_ids:     hdl:21.14100/150d6780-c503-4923-beb4-265e947d148...
    version_id:              v20190710
    intake_esm_varname:      ['thetao']
    intake_esm_dataset_key:  CMIP.MPI-M.MPI-ESM1-2-HR.historical.Omon.gn

In [5]:
test_grid = Grid(CFSR_thetao)
test_grid

<xgcm.Grid>
X Axis (periodic, boundary=None):
  * center   lon
Y Axis (periodic, boundary=None):
  * center   lat
Z Axis (periodic, boundary=None):
  * center   lev
T Axis (periodic, boundary=None):
  * center   time

In [6]:
cmip6_interp = test_grid.interp(ds, axis='Z')

KeyError: 'center'

### Try regrid with WOA data

In [7]:
URL = 'https://www.ncei.noaa.gov/thredds-ocean/dodsC/ncei/woa/temperature/decav81B0/0.25/woa18_decav81B0_t00_04.nc'
WOA_temp = xr.open_dataset(URL,engine='netcdf4',decode_times=False)

In [8]:
test_grid = Grid(WOA_temp)
test_grid

<xgcm.Grid>
X Axis (periodic, boundary=None):
  * center   lon
Y Axis (periodic, boundary=None):
  * center   lat
Z Axis (periodic, boundary=None):
  * center   depth
T Axis (periodic, boundary=None):
  * center   time

In [10]:
cmip6_interp = test_grid.interp(ds, axis='Z')

KeyError: "None of the DataArray's dims Frozen(SortedKeysDict({'x': 802, 'y': 404, 'lev': 40, 'bnds': 2, 'member_id': 1, 'time': 1980, 'vertex': 4})) were found in axis coords."